### 日常工作中，我们经常需要对表格中某一列或某几列中的，用分隔符分割的字符进行统计分析
    
### pandas内置了很多有用的string处理方法，下面让我们通过一些示例来进行学习:

- 通过今天的分享，你很有可能掌握下面的内容：
	- 修改字符的大小写，例如手写字母大写
	- 获取字符串的长度等属性
	- 根据自定义的字符模式提取字符串
	- 分割字符串
	- 统计每个字符串出现的次数

In [20]:
# 导入必要的包
import numpy as np
import pandas as pd

In [21]:
# 自行构造series数据 ['peter','paul','hj',None,'MARY','Mark']
names = pd.Series(data=['peter','paul','hj',None,'MARY','Mark'])
names

0    peter
1     paul
2       hj
3     None
4     MARY
5     Mark
dtype: object

In [22]:
# 首字母大写
names.str.capitalize()

0    Peter
1     Paul
2       Hj
3     None
4     Mary
5     Mark
dtype: object

In [23]:
# 全部字母大写
names.str.upper()

0    PETER
1     PAUL
2       HJ
3     None
4     MARY
5     MARK
dtype: object

In [24]:
# 判断是不是全部小写
names.str.islower()

0     True
1     True
2     True
3     None
4    False
5    False
dtype: object

In [25]:
# 字符的长度
names.str.len()

0    5.0
1    4.0
2    2.0
3    NaN
4    4.0
5    4.0
dtype: float64

#### 以上只是字符类型的一些基础方法，非常简单，也很使用，下面介绍一些稍微复杂的字符匹配处理的方法

In [26]:
# 自行构造series数据 
monte = pd.Series(['Graham Chapman','jhon cleese','Terry Gilliam','Eric Idle','Terry Jones','Micheal Palin'])
monte

0    Graham Chapman
1       jhon cleese
2     Terry Gilliam
3         Eric Idle
4       Terry Jones
5     Micheal Palin
dtype: object

In [27]:
# 判断一个字符串是否以某个字母开头
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

In [30]:
# 以指定分隔符，切分字符串
monte.str.split(' ').to_list()
monte.str.split(' ').to_dict()

{0: ['Graham', 'Chapman'],
 1: ['jhon', 'cleese'],
 2: ['Terry', 'Gilliam'],
 3: ['Eric', 'Idle'],
 4: ['Terry', 'Jones'],
 5: ['Micheal', 'Palin']}

In [32]:
# 获取切分后的指定位置的字符
monte.str.split(' ').str.get(-1)

0    Chapman
1     cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

In [36]:
# 找到第一个，匹配正则模式的字符串，如果有分组，则按分组显示 r'^([A-Za-z]+)\s+(.*)'
res_df = monte.str.extract(r'^([A-Za-z]+)\s+(.*)')
res_df

,0,1
0,Graham,Chapman
1,jhon,cleese
2,Terry,Gilliam
3,Eric,Idle
4,Terry,Jones
5,Micheal,Palin


In [35]:
type(res_df)

pandas.core.frame.DataFrame

In [37]:
# 找到字符串中所有匹配给定模式的字符 r'^[^AEIOU].*[^aeiou]$'
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Micheal Palin]
dtype: object

#### 下面通过一个简单的例子，分享一个字符串统计分析的常用方法，也许工作中会用到

In [38]:
# 构建原始数据
# 'info':['B|C|D','B|D','A|C','B|D','B|C','B|C|D']
# 'name':['Graham Chapman','jhon cleese','Terry Gilliam','Eric Idle','Terry Jones','Micheal Palin']
full_monte = pd.DataFrame({'name':['Graham Chapman','jhon cleese','Terry Gilliam','Eric Idle','Terry Jones','Micheal Palin'],'info':['B|C|D','B|D','A|C','B|D','B|C','B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,jhon cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Micheal Palin,B|C|D


#### 目标：对info列完成词频统计

In [40]:
# 使用'|'分割字符串，并转换为列表
info_se = full_monte['info']
info_li = info_se.str.split('|').to_list()
info_li

[['B', 'C', 'D'],
 ['B', 'D'],
 ['A', 'C'],
 ['B', 'D'],
 ['B', 'C'],
 ['B', 'C', 'D']]

In [44]:
# 获取列表中存在哪些类别
cat_li = sorted(list(set([ i for j in info_li for i in j ])))
cat_li

['A', 'B', 'C', 'D']

In [45]:
# 构建列为每个种类，行数为原数据总行数的全0 DateFrame
df_zero = pd.DataFrame(np.zeros((len(info_li),len(cat_li)),dtype=np.int),columns=cat_li)
df_zero

,A,B,C,D
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0


In [46]:
# 逐行统计
for i,info in enumerate(info_li):
    df_zero.loc[i,info] = 1
df_zero

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


In [47]:
df_zero.sum(axis=0)

A    1
B    5
C    4
D    4
dtype: int64

In [48]:
info_se

0    B|C|D
1      B|D
2      A|C
3      B|D
4      B|C
5    B|C|D
Name: info, dtype: object

In [49]:
# 逐列统计
for cat in cat_li:
    df_zero[cat][info_se.str.contains(cat)] = 1
df_zero

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


In [50]:
df_zero.sum()

A    1
B    5
C    4
D    4
dtype: int64

In [51]:
# 一步完成字符出现频率统计，又称one-hot编码
res_df = full_monte['info'].str.get_dummies('|')
res_df

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


In [52]:
res_df.sum()

A    1
B    5
C    4
D    4
dtype: int64